### ✅ Face Detection
- Face Tracking 진행 예정!

### ✅ 1. 카메라 인스턴스 생성

In [1]:
from jetbot import Camera # 카메라 관련 라이브러리
from jetbot import bgr8_to_jpeg # bgr type -> jpeg 변환

camera = Camera.instance(width = 720, height = 720)

In [ ]:
# 오류?
# 1) 왼쪽 정지버튼 -> 전체 커널 셧다운
# 2) 커널 restart
# 3) 재부팅
# 4) 카메라 접촉 불량 ...

### ✅ 2. 카메라 프리뷰 생성

In [2]:
import ipywidgets.widgets as widgets #파이썬에서 위젯을 활용할 수 있는 라이브러리
from IPython.display import display #실제 이미지를 display할 수 있는 라이브러리

detect_face = widgets.Image(width = 300, height = 300, format = 'jpeg') # 인식될 공간

display(detect_face)

Image(value=b'', format='jpeg', height='300', width='300')

In [ ]:
# 카메라 테스트

while 1:
    detect_face.value = bgr8_to_jpeg(camera.value) # 카메라 값을 변환시킨다

### ✅ 학습된 모델을 가져오자!! 

- https://github.com/opencv/opencv/tree/master/data/haarcascades

In [2]:
# curl 명령어 통해서 링크로 학습된 데이터 가져오기!
# 터미널창 sudo apt-get install curl
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  908k  100  908k    0     0   386k      0  0:00:02  0:00:02 --:--:--  629k


In [3]:
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

face_x = face_y = face_w = face_h = 0 # 얼굴을 표시할 좌표를 0으로 초기화

In [ ]:
while 1:
    frame = camera.value
    
    cv2.resize(frame, (300, 300)) # 학습된 데이터의 이미지가 300*300 픽셀
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백으로 변경
    
    faces = face_cascade.detectMultiScale(gray) # 흑백 이미지 내에서 얼굴 검출
    
    if len(faces) > 0 : # 만약 검출된게 있다면
        (face_x, face_y, face_w, face_h) = faces[0] # 사진에 좌표 생성
        
        # 직사각형 그리기!
        cv2.rectangle(frame, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 0, 255), 4) # 직사각형 생성 # (0, 0, 255) -> 빨강
    
    detect_face.value = bgr8_to_jpeg(frame)

### ✅ 눈 검출!
- ROT(Region of Interest)

In [1]:
# 눈 관련 학습 모델 가져오기!
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_eye.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  333k  100  333k    0     0   183k      0  0:00:01  0:00:01 --:--:--  322k


In [4]:
import cv2

# 눈 관련 cascade 모델 생성!

eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# 눈 좌표 초기화

eye_x = eye_y = eye_w = eye_h = 0

In [ ]:
while 1:
    frame = camera.value
    
    cv2.resize(frame, (300, 300)) # 학습된 데이터의 이미지가 300*300 픽셀
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백으로 변경
    
    faces = face_cascade.detectMultiScale(gray) # 흑백 이미지 내에서 얼굴 검출
    
    if len(faces) > 0 : # 만약 검출된게 있다면
        (face_x, face_y, face_w, face_h) = faces[0] # 사진에 좌표 생성
        
        # 직사각형 그리기!
        
        cv2.rectangle(frame, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 0, 255), 4)
        # 직사각형 생성 # (0, 0, 255) -> 빨강
        
        # 눈이 검출된다 -> 먼저 얼굴이 검출되어야 함
        
        roi_region = gray[face_y : face_y + face_h, face_x : face_x + face_w]
                     # -> gray 전체 프레임에서 얼굴 영역만 슬라이싱
            
        roi_color = frame[face_y : face_y + face_h, face_x : face_x + face_w]
                    # -> 눈을 표시할 수 있는 도형을 컬러 프레임에 찍기 위함! # 원본 -> frame
        
        eyes = eye_cascade.detectMultiScale(roi_region) # 눈 검출
        
        # 눈이 검출되었다면! 
        
        for eye_x, eye_y, eye_w, eye_h in eyes : # 눈이라고 인식된 모든 공간
            cv2.rectangle(roi_color, (eye_x, eye_y), (eye_x + eye_w, eye_y + eye_h), (255, 0, 0), 2)
            # (255, 0, 0) -> 파랑
    
    detect_face.value = bgr8_to_jpeg(frame)